Ref: https://github.com/tungk/OED

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tqdm.notebook import tqdm
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, LSTMCell, GRU, GRUCell, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector, TimeDistributed

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class SkipRNN(tf.keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.get_initial_state = getattr(
            self.cell, "get_initial_state", self.fallback_initial_state)
    def fallback_initial_state(self, inputs):
        return [tf.zeros([self.cell.state_size], dtype=inputs.dtype)]
    @tf.function
    def call(self, inputs, states=None):
        states = self.get_initial_state(inputs) if states == None else states

        outputs = tf.zeros(shape=[self.cell.output_size], dtype=inputs.dtype)
        outputs, states = self.cell(inputs, states)

        return outputs, states
    
def S_RNN(X_train):
    tf.keras.backend.clear_session()

    sparseness_weights = [(0, 1), (1, 0), (1, 1)]
    BATCH_SIZE = 128
    N, N_LAYERS, N_UNITS = 40, 1, 8

    X_train_reverse = np.flip(X_train, axis=1)
    seq_length, dim = X_train.shape[1], X_train.shape[2]

    en_input = Input(shape=[seq_length, dim])
    X = GaussianNoise(0.5)(en_input)
    initial_states = tf.zeros([BATCH_SIZE, N_UNITS])

    shared_latents = []
    for i in range(N):
        prev_states = []
        skip_length = np.random.randint(low=2, high=10, size=1)[0]
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0]
        w = w1 + w2

        for t in range(seq_length):
            Xt = Lambda(lambda x: x[:, t, :])(X)
            if t == 0:
                O, H = SkipRNN(GRUCell(N_UNITS))(Xt)
            else:
                if t - skip_length >= 0:
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, prev_states[t-1])
                else:
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, prev_states[t-1])

            prev_states.append(H)
        shared_latents.append(H)

    de_outputs = []
    de_input = Concatenate()(shared_latents)
    D = Dense(dim, kernel_regularizer=tf.keras.regularizers.l1(0.005))(de_input)

    for i in range(N):
        Y_i = []
        prev_states = []
        skip_length = np.random.randint(low=2, high=10, size=1)[0]
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0]
        w = w1 + w2

        for t in range(seq_length):
            if t == 0:
                y = Dense(dim)(D)
                _, H = SkipRNN(GRUCell(dim))(y, D) # y_t
            else:
                if t - skip_length >= 0:
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], states) # y_t-1 --> y_1
                else:
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], prev_states[t-1]) # y_t-1 --> y_1

            Y_i.append(y)
            prev_states.append(H)

        Y_i = Concatenate()(Y_i)
        Y_i = Reshape([seq_length, dim])(Y_i)
        de_outputs.append(Y_i)

    model = Model(inputs=en_input, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

    history = model.fit(X_train, [X_train_reverse for _ in range(N)], batch_size=BATCH_SIZE, epochs=50, validation_split=0.3, verbose=0, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
    return model

### Yahoo S5

In [4]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [5]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.9999999000000052 0.0 0.9999999490291289
yahoo_A1 0.9999999166666703 0.6666666055555603 0.9999999656957941
yahoo_A1 0.9090908429752097 0.7547169388267891 0.9166666504722225
yahoo_A1 0.9999998500000123 0.0 0.9999998990476292
yahoo_A1 0.666666577777783 0.04772726743317393 0.954326864346805
yahoo_A1 0.6666666000000033 0.24999997083333655 0.9951922564950103
yahoo_A1 0.9999999166666703 0.6666666055555603 0.999999965714287
yahoo_A1 0.9999999466666691 0.9666666518500433 0.9999999953333334
yahoo_A1 0.9830507941396176 0.9626856655529366 0.9982222175638518
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790099013
yahoo_A1 0.9999999333333361 0.8333332883333358 0.9999999823232326
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790000004
yahoo_A1 0.9999999437500025 0.8749999813053398 0.9999999926264045
yahoo_A1 0.05555555005144069 0.004854369563728194 -0.0
yahoo_A1 0.9090908429752097 0.718333289169447 0.9961164839649359
yahoo_A1 0.9999999409090937 0.909090878331695 0.9999999

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.03278688196721328 0.006493506464253767 0.37704914415815993
yahoo_A3 0.12121210964187425 0.04465169045416881 0.419822448076577
yahoo_A3 0.9090908429752097 0.7135964510388219 0.9802259715327655
yahoo_A3 0.03174602857142873 0.004065040847075319 -0.0
yahoo_A3 0.6666666133333363 0.5402099349125873 0.878167630265388
yahoo_A3 0.0624999938476567 0.007396145110226585 0.18333332109722314
yahoo_A3 0.17647057188581453 0.03959920951947576 0.22184471982622514
yahoo_A3 0.09230768336094745 0.014346359329940009 0.3711864171404748
yahoo_A3 0.4999999500000025 0.19444739502681022 0.6584745513360396
yahoo_A3 0.14925371730897866 0.06511381863905998 0.49647729613751157
yahoo_A3 0.14925371730897866 0.05337387259843753 0.48172165595666505
yahoo_A3 0.33333330000000166 0.15561263446942417 0.5034391459655898
yahoo_A3 0.09090907355372142 0.024404760019650737 0.7540982899556639
yahoo_A3 0.17647057188581453 0.07089918645576541 0.3671108369258267
yahoo_A3 0.04724408980097991 0.004098361191078604 -0.0
yahoo

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.09230768336094745 0.006410257201072274 0.01694915083309413
yahoo_A4 0.03252032195121968 0.008483705555738087 0.29918030221378894
yahoo_A4 0.0624999938476567 0.0048543696516308256 0.14999998633333464
yahoo_A4 0.0624999938476567 0.005434783289289459 0.2416666452152798
yahoo_A4 0.5882352373702464 0.4262398962682289 0.7741687080807624
yahoo_A4 0.19354835088450145 0.07344745516454199 0.5693247045287451
yahoo_A4 0.0624999938476567 0.004166667438873992 0.008333332493055609
yahoo_A4 0.07751937227330145 0.0041666676308108625 -0.0
yahoo_A4 0.2142856790816377 0.06453928235423739 0.5227401020706537
yahoo_A4 0.2857142285714372 0.08234987053107126 0.7312499811093756
yahoo_A4 0.12121210964187425 0.024174714613285575 0.1530890771696312
yahoo_A4 0.0624999938476567 0.024151800089795634 0.5249999761597235
yahoo_A4 0.07692305591716472 0.02420646232952301 0.46597220435706094
yahoo_A4 0.11538459452663061 0.042453302888235914 0.5573446201651509
yahoo_A4 0.03174602857142873 0.004065040847075319 -0.

In [6]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.696685,0.436189,0.758318
yahoo_A2,0.699042,0.364772,0.884585
yahoo_A3,0.277608,0.123376,0.507688
yahoo_A4,0.211283,0.078792,0.412303


### NASA

In [7]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [8]:
for loader in [load_nasa]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.2295534087061448 0.11003824939559584 0.4139143217842644
D2 0.19498578202833744 0.5212749842587658 0.5131570625923163


In [9]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.229553,0.110038,0.413914
D2,0.194986,0.521275,0.513157


In [ ]:
0.229553	0.110038	0.413914
0.194986	0.521275	0.513157

### SMD

In [10]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [11]:
for loader in [load_smd]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')
      
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.17627464443920743 0.5532991977145021 0.5087209301507045
smd 0.5399999496600046 0.46820320093451584 0.9174606284948894
smd 0.12933020402692222 0.11520692326299273 0.5464818930171673
smd 0.10659893474969488 0.10887758263220347 0.5370828666694738
smd 0.6551723644470901 0.6233629873761043 0.900400461625784
smd 0.8511627415400245 0.8494507811057643 0.9037465181899358
smd 0.20705270169868853 0.19790102394933234 0.5611360991457136
smd 0.2713178052340625 0.2133763935425203 0.6936957365759141
smd 0.4599302638389496 0.3860837746060086 0.7421939431034245
smd 0.21837868410447125 0.20441080089401778 0.5089180502256726
smd 0.5555555107709786 0.6110920331500617 0.7054146667385478
smd 0.3148147876673548 0.3347334135704506 0.7715728218721432
smd 0.29268287755635697 0.2543698716012064 0.7753698752768575
smd 0.45751630398564924 0.38682729480761424 0.6569959209854374
smd 0.6814158814550898 0.6343105371593585 0.902700471362889
smd 0.9499999477500025 0.894218436435123 0.9760487986827799
smd 0.51519996

In [12]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.449216,0.447218,0.747684


### ECG

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [14]:
for loader in [load_ecg]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]

        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.5418718764250564 0.49635816339280703 0.5546733482135279
D2 0.35483866012834536 0.2950869984173311 0.5365307914264048
D3 0.17999996204889684 0.06557002340874174 0.5488697064048228
D4 0.2697946900319091 0.1588018514120823 0.6171449229812186
D5 0.40196073884083555 0.25007229833408623 0.7919853980999886
D6 0.1571194617650775 0.07832233770163353 0.410831736033611
D7 0.05803431526519908 0.020346158266264663 0.5515446764397027
D8 0.18800646397387558 0.086568664043101 0.4868127184723462
D9 0.37051403375627384 0.1968364560378836 0.47576716032869243


In [15]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.541872,0.496358,0.554673
D2,0.354839,0.295087,0.536531
D3,0.180000,0.065570,0.548870
D4,0.269795,0.158802,0.617145
D5,0.401961,0.250072,0.791985
D6,0.157119,0.078322,0.410832
D7,0.058034,0.020346,0.551545
D8,0.188006,0.086569,0.486813
D9,0.370514,0.196836,0.475767


### Power Demand

In [16]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [17]:
for loader in [load_power_demand]:
    datasets = loader(16, 8)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]

        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.2748814926169693 0.09598536182653103 0.3188498649795368


In [18]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.274881,0.095985,0.31885


### 2D Gesture

In [19]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [20]:
for loader in [load_gesture]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]

        model = S_RNN(X_train)
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test)]
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True, scoring='square_median')

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.4640135051186586 0.28363514646252697 0.5646321761245787


In [21]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.464014,0.283635,0.564632
